In [1]:
from scipy.optimize import curve_fit
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import warnings
import ruptures as rpt
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_absolute_error as MAE
plt.style.use('ggplot')
warnings.filterwarnings("ignore")

In [2]:
def f(X, K, T):
    x, t = X    
    return K * (1 - np.exp(-t / T)) * x

In [3]:
def make_data(x, y, x_cp_list, y_cp_list, D):
    data = []
    for cp in x_cp_list[:-1]:
        LR_y = x[cp-1:cp+2]
        LR_x = np.array(range(1, len(LR_y) + 1)).reshape(-1, 1)

        old_theta = LR().fit(LR_x, LR_y).coef_[0]
        a = 2
        while True:
            LR_y = x[cp-a:cp+a+1]
            LR_x = np.array(range(1, len(LR_y) + 1)).reshape(-1, 1)
            new_theta = LR().fit(LR_x, LR_y).coef_[0]

            if (new_theta * old_theta < 0) or (abs(new_theta) - abs(old_theta)) / abs(old_theta) > eps or (a >= cp):
                a -= 1
                break    
            a += 1
            old_theta = new_theta    

        t = 1
        for x_val, y_val in zip(x[cp-a:cp+a+1], y[cp-a+D:cp+a+1+D]):
            try:
                record = [t, x_val, y_val]
                data.append(record)
            except:
                pass
            t+=1
            
    for cp in y_cp_list[:-1]:
        LR_y = y[cp-1:cp+2]
        LR_x = np.array(range(1, len(LR_y) + 1)).reshape(-1, 1)

        old_theta = LR().fit(LR_x, LR_y).coef_[0]
        a = 2
        while True:
            LR_y = x[cp-a:cp+a+1]
            LR_x = np.array(range(1, len(LR_y) + 1)).reshape(-1, 1)
            new_theta = LR().fit(LR_x, LR_y).coef_[0]

            if (new_theta * old_theta < 0) or (abs(new_theta) - abs(old_theta)) / abs(old_theta) > eps or (a >= cp):
                a -= 1
                break    
            a += 1
            old_theta = new_theta    
        
        t = 1
        for x_val, y_val in zip(x[cp-a-D:cp+a+1-D], y[cp-a:cp+a+1]):
            try:
                record = [t, x_val, y_val]
                data.append(record)        
            except:
                pass
            t += 1
        
    data = pd.DataFrame(data, columns = ["t", "x", "y"])
    return data

In [4]:
def evaluate(train_data, test_data):
    X = train_data[['x', 't']].T.values
    y = train_data['y'].values    
    
    X_test = test_data[['x', 't']].T.values
    y_test = test_data['y'].values
    
    K_hat, T_hat = curve_fit(f, X, y)[0]    
    y_hat = f(X_test, K_hat, T_hat)
    
    return MAE(y_test, y_hat)

In [5]:
s_train_df = pd.read_excel("Plant S_Data추가.xlsx", sheet_name = "Temp_Data04").iloc[:1440*5]
s_test_df = pd.read_excel("Plant S_Data추가.xlsx", sheet_name = "Temp_Data03").iloc[:1440*5]

In [6]:
result = []
num_iter = 0
for train_df, test_df, idx in zip([s_train_df], [s_test_df], ["s"]):
    for var in ["DV", "MV"]:
        for algorithm in [rpt.Pelt, rpt.KernelCPD]:
            for D in [0, 3, 5, 10]:
                for ms in [5, 10, 15, 20]:
                    for eps in [0.01, 0.05, 0.1]:
                        for pen in [3]:
                            for kernel in ["linear", "rbf", "cosine"]:                        
                                x_train = train_df[var].values
                                y_train = train_df['CV'].values

                                x_test = test_df[var].values
                                y_test = test_df['CV'].values

                                if algorithm.__name__ == "Pelt":
                                    x_train_cp_list = algorithm(min_size = ms).fit(x_train).predict(pen = pen)
                                    y_train_cp_list = algorithm(min_size = ms).fit(y_train).predict(pen = pen)
                                    x_test_cp_list = algorithm(min_size = ms).fit(x_test).predict(pen = pen)
                                    y_test_cp_list = algorithm(min_size = ms).fit(y_test).predict(pen = pen)
                                    try:
                                        train_data = make_data(x_train, y_train, x_train_cp_list, y_train_cp_list, D)
                                        test_data = make_data(x_test, y_test, x_test_cp_list, y_test_cp_list, D)
                                        score = evaluate(train_data, test_data)
                                        record = [idx, var, algorithm.__name__, kernel, D, ms, eps, len(x_train_cp_list), len(x_test_cp_list), len(y_train_cp_list), len(y_test_cp_list), score]   
                                        result.append(record)
                                    except:
                                        pass
                                    num_iter += 1
                                    print(num_iter, "/1536")

                                    break

                                else:
                                    x_train_cp_list = algorithm(min_size = ms, kernel = kernel).fit(x_train).predict(pen = pen)
                                    y_train_cp_list = algorithm(min_size = ms, kernel = kernel).fit(y_train).predict(pen = pen)
                                    x_test_cp_list = algorithm(min_size = ms, kernel = kernel).fit(x_test).predict(pen = pen)
                                    y_test_cp_list = algorithm(min_size = ms, kernel = kernel).fit(y_test).predict(pen = pen)
                                    
                                    try:
                                        train_data = make_data(x_train, y_train, x_train_cp_list, y_train_cp_list, D)
                                        test_data = make_data(x_test, y_test, x_test_cp_list, y_test_cp_list, D)
                                        score = evaluate(train_data, test_data)
                                        record = [idx, var, algorithm.__name__, kernel, D, ms, eps, len(x_train_cp_list), len(x_test_cp_list), len(y_train_cp_list), len(y_test_cp_list), score]   
                                        result.append(record)
                                    except:
                                        pass        
                                    
                                    num_iter += 1                                    
                                    print(num_iter, "/1536")

1 /1536
2 /1536
3 /1536
4 /1536
5 /1536
6 /1536
7 /1536
8 /1536
9 /1536
10 /1536
11 /1536
12 /1536
13 /1536
14 /1536
15 /1536
16 /1536
17 /1536
18 /1536
19 /1536
20 /1536
21 /1536
22 /1536
23 /1536
24 /1536
25 /1536
26 /1536
27 /1536
28 /1536
29 /1536
30 /1536
31 /1536
32 /1536
33 /1536
34 /1536
35 /1536
36 /1536
37 /1536
38 /1536
39 /1536
40 /1536
41 /1536
42 /1536
43 /1536
44 /1536
45 /1536
46 /1536
47 /1536
48 /1536
49 /1536
50 /1536
51 /1536
52 /1536
53 /1536
54 /1536
55 /1536
56 /1536
57 /1536
58 /1536
59 /1536
60 /1536
61 /1536
62 /1536
63 /1536
64 /1536
65 /1536
66 /1536
67 /1536
68 /1536
69 /1536
70 /1536
71 /1536
72 /1536
73 /1536
74 /1536
75 /1536
76 /1536
77 /1536
78 /1536
79 /1536
80 /1536
81 /1536
82 /1536
83 /1536
84 /1536
85 /1536
86 /1536
87 /1536
88 /1536
89 /1536
90 /1536
91 /1536
92 /1536
93 /1536
94 /1536
95 /1536
96 /1536
97 /1536
98 /1536
99 /1536
100 /1536
101 /1536
102 /1536
103 /1536
104 /1536
105 /1536
106 /1536
107 /1536
108 /1536
109 /1536
110 /1536
111 /153

832 /1536
833 /1536
834 /1536
835 /1536
836 /1536
837 /1536
838 /1536
839 /1536
840 /1536
841 /1536
842 /1536
843 /1536
844 /1536
845 /1536
846 /1536
847 /1536
848 /1536
849 /1536
850 /1536
851 /1536
852 /1536
853 /1536
854 /1536
855 /1536
856 /1536
857 /1536
858 /1536
859 /1536
860 /1536
861 /1536
862 /1536
863 /1536
864 /1536
865 /1536
866 /1536
867 /1536
868 /1536
869 /1536
870 /1536
871 /1536
872 /1536
873 /1536
874 /1536
875 /1536
876 /1536
877 /1536
878 /1536
879 /1536
880 /1536
881 /1536
882 /1536
883 /1536
884 /1536
885 /1536
886 /1536
887 /1536
888 /1536
889 /1536
890 /1536
891 /1536
892 /1536
893 /1536
894 /1536
895 /1536
896 /1536
897 /1536
898 /1536
899 /1536
900 /1536
901 /1536
902 /1536
903 /1536
904 /1536
905 /1536
906 /1536
907 /1536
908 /1536
909 /1536
910 /1536
911 /1536
912 /1536
913 /1536
914 /1536
915 /1536
916 /1536
917 /1536
918 /1536
919 /1536
920 /1536
921 /1536
922 /1536
923 /1536
924 /1536
925 /1536
926 /1536
927 /1536
928 /1536
929 /1536
930 /1536
931 /1536


In [7]:
result = pd.DataFrame(result, columns = ["data", "x_var", "algorithm", "kernel", "D", "ms", "eps", "x_train_cp_길이", "x_test_cp_길이", "y_train_cp_길이", "y_test_cp_길이", "score"])

In [9]:
result = result.drop_duplicates(subset = ['x_var', 'algorithm', 'kernel', 'D', 'ms', 'eps'])

In [10]:
result.to_csv("유요셉_실험결과_s.csv", encoding = "euc-kr")

In [11]:
mv_result = result.loc[result['x_var'] == 'MV']
dv_result = result.loc[result['x_var'] == 'DV']

In [12]:
display(mv_result.sort_values(by = 'score').iloc[:10])
display(dv_result.sort_values(by = 'score').iloc[:10])

,data,x_var,algorithm,kernel,D,ms,eps,x_train_cp_길이,x_test_cp_길이,y_train_cp_길이,y_test_cp_길이,score
793,s,MV,KernelCPD,rbf,0,10,0.01,33,19,66,76,45.264157
769,s,MV,KernelCPD,rbf,0,5,0.01,35,19,81,80,45.520872
841,s,MV,KernelCPD,rbf,0,20,0.01,32,19,55,72,45.775182
784,s,MV,KernelCPD,linear,0,5,0.10,473,35,195,206,45.834073
776,s,MV,KernelCPD,linear,0,5,0.05,473,35,195,206,45.849358
817,s,MV,KernelCPD,rbf,0,15,0.01,32,19,60,73,45.896156
785,s,MV,KernelCPD,rbf,0,5,0.10,35,19,81,80,45.936669
1081,s,MV,KernelCPD,rbf,10,10,0.01,33,19,66,76,46.000965
985,s,MV,KernelCPD,rbf,5,10,0.01,33,19,66,76,46.033829
833,s,MV,KernelCPD,rbf,0,15,0.10,32,19,60,73,46.043436


,data,x_var,algorithm,kernel,D,ms,eps,x_train_cp_길이,x_test_cp_길이,y_train_cp_길이,y_test_cp_길이,score
361,s,DV,KernelCPD,rbf,3,20,0.01,114,76,55,72,3.987124
457,s,DV,KernelCPD,rbf,5,20,0.01,114,76,55,72,3.988567
265,s,DV,KernelCPD,rbf,0,20,0.01,114,76,55,72,3.990004
553,s,DV,KernelCPD,rbf,10,20,0.01,114,76,55,72,3.994029
44,s,DV,Pelt,linear,0,20,0.10,87,109,108,150,4.094297
216,s,DV,KernelCPD,linear,0,10,0.01,100,127,163,188,4.148142
312,s,DV,KernelCPD,linear,3,10,0.01,100,127,163,188,4.149616
92,s,DV,Pelt,linear,3,20,0.10,87,109,108,150,4.150099
408,s,DV,KernelCPD,linear,5,10,0.01,100,127,163,188,4.153172
140,s,DV,Pelt,linear,5,20,0.10,87,109,108,150,4.154886


In [13]:
mv_result.groupby(['algorithm', 'kernel'])['score'].mean().reset_index()

,algorithm,kernel,score
0,KernelCPD,linear,47.525187
1,KernelCPD,rbf,46.326916
2,Pelt,linear,49.346287


In [14]:
dv_result.groupby(['algorithm', 'kernel'])['score'].mean().reset_index()

,algorithm,kernel,score
0,KernelCPD,linear,4.293719
1,KernelCPD,rbf,4.341620
2,Pelt,linear,4.400131


In [15]:
mv_result.groupby(['D'])['score'].mean().reset_index()

,D,score
0,0,47.305815
1,3,47.877064
2,5,47.883309
3,10,47.864999


In [16]:
dv_result.groupby(['D'])['score'].mean().reset_index()

,D,score
0,0,4.327684
1,3,4.345038
2,5,4.348035
3,10,4.359868


In [17]:
mv_result.groupby(['ms'])['score'].mean().reset_index()

,ms,score
0,5,47.221883
1,10,47.544368
2,15,48.032792
3,20,48.132144


In [18]:
dv_result.groupby(['ms'])['score'].mean().reset_index()

,ms,score
0,5,4.371485
1,10,4.324087
2,15,4.386966
3,20,4.298089


In [19]:
mv_result.groupby(['eps'])['score'].mean().reset_index()

,eps,score
0,0.01,48.184318
1,0.05,47.632912
2,0.10,47.381161


In [20]:
dv_result.groupby(['eps'])['score'].mean().reset_index()

,eps,score
0,0.01,4.270740
1,0.05,4.337487
2,0.10,4.427242
